In [1]:
import requests
import pandas as pd
from io import StringIO

## Deliverables #1

In [2]:
import requests
import pandas as pd
from io import StringIO

def extract(url):
    response = requests.get(url)
    response.raise_for_status() 
    data = StringIO(response.text)
    df = pd.read_csv(data)
    return df

#Electric Vehicle Population Data
url = 'https://data.wa.gov/api/views/f6w7-q2d2/rows.csv?accessType=DOWNLOAD'

# Extract the data into pandas dataframe
ev_df = extract(url)
print(ev_df.head())  # Display the first few rows of the dataframe


   VIN (1-10)    County     City State  Postal Code  Model Year   Make  \
0  WAUTPBFF4H      King  Seattle    WA      98126.0        2017   AUDI   
1  WAUUPBFF2J  Thurston  Olympia    WA      98502.0        2018   AUDI   
2  5YJSA1E22H  Thurston    Lacey    WA      98516.0        2017  TESLA   
3  1C4JJXP62M  Thurston   Tenino    WA      98589.0        2021   JEEP   
4  5YJ3E1EC9L    Yakima   Yakima    WA      98902.0        2020  TESLA   

      Model                   Electric Vehicle Type  \
0        A3  Plug-in Hybrid Electric Vehicle (PHEV)   
1        A3  Plug-in Hybrid Electric Vehicle (PHEV)   
2   MODEL S          Battery Electric Vehicle (BEV)   
3  WRANGLER  Plug-in Hybrid Electric Vehicle (PHEV)   
4   MODEL 3          Battery Electric Vehicle (BEV)   

  Clean Alternative Fuel Vehicle (CAFV) Eligibility  Electric Range  \
0             Not eligible due to low battery range              16   
1             Not eligible due to low battery range              16   
2          

In [3]:
# Function to create a data dictionary
def create_data_dictionary(df):
    data_dict = []
    for column in df.columns:
        data_dict.append({
            'Column Name': column,
            'Data Type': df[column].dtype,
            'Sample Data': df[column].dropna().unique()[:5]  # Show up to 5 unique values
        })
    return pd.DataFrame(data_dict, columns=['Column Name', 'Data Type', 'Sample Data'])

# Create data dictionary
dictionary = create_data_dictionary(ev_df)

# Export data dictionary to Excel
file_name = "data_dictionary.xlsx"
dictionary.to_excel(file_name, index=False)
print(f'Data dictionary exported to {file_name}')

Data dictionary exported to data_dictionary.xlsx


## Deliverables #2

In [9]:
from google.cloud import storage
from datetime import datetime
from google.cloud import bigquery
import pandas as pd
from io import StringIO
from google.oauth2 import service_account
import pandas_gbq
import json
from pandas_gbq import to_gbq
import os

In [5]:
# Define the storage function
def upload_dataframe_to_gcs(bucket_name, df, file_name, project_id):
    now = datetime.now()

    formatted_date = now.strftime("%Y-%m-%d")
    destination_folder = f"{formatted_date}"

    formatted_time = now.strftime("%Y%m%d%H%M%S")
    file_name = f"{file_name}_{formatted_time}.csv"

    csv_string = df.to_csv(index=False)

    storage_client = storage.Client(project=project_id)
    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(f"{destination_folder}/{file_name}")
    blob.upload_from_string(csv_string, content_type='text/csv')

    print(f"DataFrame uploaded to {bucket_name}/{destination_folder}/{file_name}.")

In [11]:
# GCP JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "prime-task-421716-f42265232751.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

prime-task-421716-f42265232751.json


In [12]:
# Push it to GCP
bucket_name = "hw1_ev_bucket"
project_id = "prime-task-421716"
upload_dataframe_to_gcs(bucket_name, ev_df, "electric_vehicle_population_data", project_id)

DataFrame uploaded to hw1_ev_bucket/2024-04-28/electric_vehicle_population_data_20240428165655.csv.
